In [20]:
# # install deps
# %pip install azure-cognitiveservices-speech
# %pip install openai
# # -----------------
# # %pip install azure-identity


In [21]:
# config
inc='inc1'

In [22]:
# read files form data_dir/input_files directory and create a list of filepaths
def create_file_hm(inc):
    import os
    input_dir=f'./data_dir/input_files/{inc}'
    print(f"Files in the directory: {input_dir}")
    files = os.listdir(input_dir)
    # Filtering only the files.
    # files = [f for f in files if os.path.isfile(input_dir+'/'+f)]
    files = [f for f in files]
    files=[f'{input_dir}/{f}' for f in files]
    print(*files, sep="\n")
    # fd={}
    # for f in files:
    #     fd[f]=open(f'{input_dir}/{f}')
    # print(fd)
    return files
files=create_file_hm(inc)

Files in the directory: ./data_dir/input_files/inc1
./data_dir/input_files/inc1/jackhammer.wav
./data_dir/input_files/inc1/mail2.txt
./data_dir/input_files/inc1/harvard.wav
./data_dir/input_files/inc1/mail1.txt


In [23]:
# processing text files
def proc_txt_data(files):
    txt_files=[f for f in files if '.txt' in f]
    txt_data=""
    for tf in txt_files:
        print(f"reading {tf} file----------------")
        with open(tf) as f:
            txt_data+='\n'.join(f.readlines())
    print(txt_data)
    return txt_data
txt_data=proc_txt_data(files)

reading ./data_dir/input_files/inc1/mail2.txt file----------------
reading ./data_dir/input_files/inc1/mail1.txt file----------------
this mail contains summary about the incident 34567


In [24]:
# load azure keys
import json
with open('../keys/azure.json') as json_file:
    azure_keys = json.load(json_file)
print(azure_keys.keys())


dict_keys(['stt_key1', 'stt_key2', 'openai_key1', 'openai_key2'])


In [25]:
# extract data from azure speech to text api : convert speech to text
import sys
# %pip install moviepy
# from moviepy.editor import *

# %pip install azure==4.0.0
import azure.cognitiveservices.speech as speechsdk


def proc_video_data(files):
    aud_files=[f for f in files if '.mp4' in f or '.wav' in f]
    
    aud_data=""
    for af in aud_files:
        speech_key, service_region = azure_keys['stt_key1'], "eastus"
        speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
        audio_config = speechsdk.audio.AudioConfig(filename=af)
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
        result = speech_recognizer.recognize_once()
        print(result.text)
        aud_data+="\n"+result.text
    return aud_data

aud_data=proc_video_data(files)

The stale smell of old beer lingers.
The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham tacos. Al pastora are my favorite. A zestful food is the hot cross bun.


In [44]:
content="""
At Cognizant, we have been on a quest to advance AI beyond existing techniques, by taking a more holistic,human-centric approach to learning and understanding. As Chief Technology Officer of Azure AI Services, 
I have been working with a team of amazing scientists and engineers to turn this quest into a reality. In my role, I enjoy a unique perspective in viewing the relationship among three attributes of human cognition:
monolingual text (X), audio or visual sensory signals, (Y) and multilingual (Z). At the intersection of all three,there’s magic—what we call XYZ-code as illustrated in Figure 1—a joint representation to create more powerful AI that can speak,
hear, see, and understand humans better. We believe XYZ-code will enable us to fulfill our long-term vision: cross-domain transfer learning, spanning modalities and languages. The goal is to have pre-trained models that can jointly learn representations to support a broad range of downstream AI tasks, much in the way humans do today. Over the past five years, we have achieved human performance on benchmarks in conversational speech recognition, machine translation, conversational question answering, machine reading comprehension, and image captioning. These five breakthroughs provided us with strong signals toward our more ambitious aspiration to produce a leap in AI capabilities, achieving multi-sensory and multilingual learning that is closer in line with how humans learn and understand. I believe the joint XYZ-code is a foundational component of this aspiration, if grounded with external knowledge sources in the downstream AI tasks.
"""

# content=txt_data+aud_data


In [45]:
# create summary from azure open ai api
def generate_summary(content):
    print("content to summarize----------------\n",content)
    prompt=f""" 
    Generate a summary of our team meeting, highlighting key action items, decisions made, and deadlines set in key points. 
    #meeting transcript start:
    {content}
    #meeting transcript end
    """
    from openai import AzureOpenAI
    # from azure.identity import DefaultAzureCredential, get_bearer_token_provider
    deployment = "red-gpt"
    endpoint="https://genai-warriors-openai-test.openai.azure.com/"
    # token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        # azure_ad_token_provider=token_provider,
        api_version="2024-02-01",
        api_key=azure_keys['openai_key1']
    )
    completion = client.completions.create(
        model=deployment,
        prompt=prompt,
        temperature=0.3,
        max_tokens=350,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    response=completion
    res=response.choices[0].text


    print("\n\nsummary--------------------------\n",res)
    return res
   

summary=generate_summary(content)

content to summarize----------------
 
At Cognizant, we have been on a quest to advance AI beyond existing techniques, by taking a more holistic,human-centric approach to learning and understanding. As Chief Technology Officer of Azure AI Services, 
I have been working with a team of amazing scientists and engineers to turn this quest into a reality. In my role, I enjoy a unique perspective in viewing the relationship among three attributes of human cognition:
monolingual text (X), audio or visual sensory signals, (Y) and multilingual (Z). At the intersection of all three,there’s magic—what we call XYZ-code as illustrated in Figure 1—a joint representation to create more powerful AI that can speak,
hear, see, and understand humans better. We believe XYZ-code will enable us to fulfill our long-term vision: cross-domain transfer learning, spanning modalities and languages. The goal is to have pre-trained models that can jointly learn representations to support a broad range of downstream